El workflow necessari i previ per obtenir els descriptors i classificadors involucra els mòduls de python següents:

	-params.py (informació de les rutes i l'arbre de directoris, i altres paràmetres globals per les funcions que posteriorment s'utilitzen.)
	-build_database.py (llegeix el nom d'arxiu de les fotografies recopilades i genera ID's er cada conjunt de dades: el d'entrenament i el de validació.
	-feature_extraction.py, get_features.py (obté el Bag of Words per a cada imatge del conjunt.)

In [ ]:
import numpy as np
import sys
# el directori superior ja s'ha afegit a la build-config.
# sys.path.insert(0,'../')
from utils.params import get_params

params = get_params()

S'assigna un ID a cada imatge i se n'extreuen els paràmetres de mètrica de la classificació.

In [3]:
import eval_classification as EC

# Get metrics
f1, precision,recall, accuracy, cm, labels = EC.eval_classification(params)

ImportError: No module named 'eval_classification'

Visualment, la matriu de conufusió dóna informació de la concordança de cada classe. Valors alts de classificació d'un element a ell mateix indiquen una bona precisió. Cal notar, que el classificador aleatori ha assignat una classe a cada imatge de manera totalment incoherent, donant lloc a una imatge sense un significat sòlid.

In [3]:
EC.plot_confusion_matrix(cm,labels,normalize=False)

Tot i així, la classe desconegut ha obtingut més bons resultats. En aparença millor, malgrat és degut al nombre més gran d'imatges que hi ha de la classe Desconegut.

In [4]:
EC.plot_confusion_matrix(cm,labels,normalize=True)

Normalitzant els resultats de cada classe amb el nombre de mostres, s'obté una matriu més representativa de l'èxit del classificador. Segueix havent-hi però, una diferència entre els valors de la classe Desconegut amb la resta. Tot i estar normalitzada, les imatges d'altres edificis ténen més possibilitats d'assignar-se a aquesta classe, ja que n'hi ha més. Donant lloc a valors lleugerament més elevats.

Seguidament s'obtenen F1, precision i recall per cada classe, així com els seus valors promig globals.

In [5]:
print("F1 per class:")
print(zip(labels,f1))
print("Precision per class:")
print(zip(labels,precision))
print("Recall per class:")
print(zip(labels,recall))

In [6]:
print("Average F1:", np.mean(f1)
print("Average Precision:", np.mean(precision))
print("Average Recall:", np.mean(recall))
print("Accuracy:", accuracy)

Resulta en uns valors baixos. Cal recordar, que l'ànim del classificador aleatori és validar el codi generat i disposar de les anotacions i prediccions dels conjunts de validació i entrenament. Estableix, però una baseline sobre la que comparar els resultats de futurs classificadors més bons.

Es procedeix a l'avaluació dels rankings havent estat prèviament computats. El mòdul rank_random.py ha generat fitxers de text de ground truth i de ranking. Se n'obté la mean average precision.

In [7]:
import eval_rankings as ER

In [8]:
ap_list = ER.eval_rankings(params)

print "Number of queries:", len(ap_list)
print "Mean Average Precision", np.mean(ap_list)

De tots els rankings, es pren la precisió mitjana global.

O, sinó mostrar la precisió d'una imatge o classe en articular.

In [9]:
# We get the true annotations of both sets. We do this in order to display this information as well.
annotation_val, annotation_train = ER.read_annotation(params)

# We get the ranking and the true class of the query
query_id = '30630-5639-13175'
query_class, ranking = ER.load_ranking(params,query_id,annotation_val)

# I made sure I was not picking an image from the "desconegut" class ...
print query_class

# The ranking is composed of the training images. It should be of size 450.
print len(ranking)

# We get the hit/miss list for the ranking
relnotrel = ER.get_hitandmiss(ranking,query_class,annotation_train)

ER.display(params,query_id,ranking,relnotrel)

De totes les classes excepte la classe desconegut, se'n poden obtenir el seu ranking i paràmetres anteriors.